In [ ]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Dropout
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Configuración del generador de imágenes para el entrenamiento
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=0,
                                   height_shift_range=0.2,
                                   width_shift_range=0.2,
                                   shear_range=0,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest',
                                   validation_split=0.2)  # Usamos un 20% de las imágenes para la validación

train_generator = train_datagen.flow_from_directory(
    'datasets/ASL_Alphabet_Dataset/asl_alphabet_train',
    target_size=(28, 28),
    color_mode='rgb',
    batch_size=32,
    class_mode='categorical',
    subset='training')  # Conjunto de datos de entrenamiento

validation_generator = train_datagen.flow_from_directory(
    'datasets/ASL_Alphabet_Dataset/asl_alphabet_train',
    target_size=(28, 28),
    color_mode='rgb',
    batch_size=32,
    class_mode='categorical',
    subset='validation')  # Conjunto de datos de validación

# Configuración del generador de imágenes para el conjunto de datos de prueba
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    'datasets/test',
    target_size=(28, 28),
    color_mode='rgb',
    batch_size=32,
    class_mode='categorical')  # Conjunto de datos de prueba

# Definición del modelo
model = Sequential()
model.add(Conv2D(128, kernel_size=(5, 5),
                 strides=1, padding='same', activation='relu', input_shape=(28, 28, 3)))
model.add(MaxPool2D(pool_size=(3, 3), strides=2, padding='same'))
model.add(Conv2D(64, kernel_size=(2, 2),
                 strides=1, activation='relu', padding='same'))
model.add(MaxPool2D((2, 2), 2, padding='same'))
model.add(Conv2D(32, kernel_size=(2, 2),
                 strides=1, activation='relu', padding='same'))
model.add(MaxPool2D((2, 2), 2, padding='same'))

model.add(Flatten())

model.add(Dense(units=512, activation='relu'))
model.add(Dropout(rate=0.25))
model.add(Dense(units=29, activation='softmax'))  # Cambié a 29 unidades para incluir las letras adicionales

model.summary()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Entrenamiento del modelo
model.fit(train_generator,
          epochs=20,
          validation_data=validation_generator,
          shuffle=True)
# Guardar el modelo entrenado
model.save('ASL2_language_model.h5')
# Evaluación del modelo en el conjunto de validación
(ls, acc) = model.evaluate(validation_generator)
print('VALIDATION ACCURACY = {}%'.format(acc * 100))

# Evaluación del modelo en el conjunto de prueba
(ls, acc) = model.evaluate(test_generator)
print('TEST ACCURACY = {}%'.format(acc * 100))
